#Langchain


**problem statement**

We are uploading a set of author details as a PDF, storing them in a folder, and loading them as a group. Now, we can retrieve all the requested details from this document


In [1]:
!pip install -q langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.0 MB/s eta 0:00:00


In [24]:


AZURE_OPENAI_ENDPOINT = "" #OpenAI Endpoint
AZURE_OPENAI_EMBEDDINGS_ENDPOINT="" #Embeddings model name = mytextembedding
AZURE_OPENAI_DEPLOYMENT_NAME = "gpt-35-turbo-16k" #GPT model name
AZURE_OPENAI_API_VERSION = "2024-02-15-preview" # OpenAI API Version
AZURE_OPENAI_API_KEY = "" #OpenAI API key


In [25]:
from langchain_openai import AzureChatOpenAI
model = AzureChatOpenAI(
        azure_endpoint=AZURE_OPENAI_ENDPOINT,
        azure_deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
        openai_api_version="2024-06-01",
        api_key=AZURE_OPENAI_API_KEY
    )



### Installing Helper Packages

In [4]:
!pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7


In [5]:
!pip install -qU langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 5.2 MB/s eta 0:00:00


In [35]:
from langchain_community.document_loaders import PyPDFLoader
import os

In [34]:
!ls

authors  sample_data


### Load and Prepare PDF

In [38]:
pdf_folder_path = "/content/authors"
documents = []
for file in os.listdir(pdf_folder_path):
  if file.endswith('.pdf'):
    pdf_path = os.path.join(pdf_folder_path, file)
    print('file',pdf_path)
    loader = PyPDFLoader(pdf_path)
    documents.extend(loader.load())
docs = documents

file /content/authors/Chetan_Bhagat.pdf
file /content/authors/William_Shakespeare.pdf
file /content/authors/Charles_Dickens.pdf
file /content/authors/Harivansh_Rai_Bachchan.pdf


In [15]:
print("\n","content:",docs[0].page_content)


 content: Khushwant Singh
Khushwant Singh receiving the National Amity
Award, in New Delhi on September 26, 2008
Born Khushal Singh
2 February 1915
Hadali, Punjab Province, British
India
(now in Punjab, Pakistan)
Died 20 March 2014 (aged 99)
New Delhi, India
Occupation Lawyer, journalist, diplomat,
writer, politician
Nationality Indian
Alma mater Government College, Lahore
(B.A.)
University of London (LL.B.)
Notable
works
The History of Sikhs
Train to Pakistan
Delhi: A Novel
The Company of Women
Truth, Love and a Little Malice:
An Autobiography
With Malice towards One and
All
Why I Supported the
Emergency: Essays and
Profiles
Khushwantnama, The Lessons
of My Life
Khushwant Singh
Khushwant Singh FKC (born Khushal Singh, 2
February 1915 – 20 March 2014) was an Indian author,
lawyer, diplomat, journalist and politician. His
experience in the 1947 Partition of India inspired him
to write Train to Pakistan in 1956 (made into film in
1998), which became his most well-known novel.[1][2]
Born

### Configuring Azure OpenAI
Notice how we're using different endpoints for Embeddings and Completions

In [26]:
!pip install -qU langchain-openai

Adding Azure Configurations to environment variables

In [39]:

os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = AZURE_OPENAI_DEPLOYMENT_NAME
os.environ["AZURE_OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION
os.environ["AZURE_OPENAI_EMBEDDINGS_ENDPOINT"] = AZURE_OPENAI_EMBEDDINGS_ENDPOINT
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_API_KEY


In [40]:
AZURE_OPENAI_EMBEDDINGS_ENDPOINT

'https://eyuse-m3y53pzv-eastus.cognitiveservices.azure.com/openai/deployments/text-embedding-3-large/embeddings?api-version=2023-05-15'

### Create Chat Model Instance

In [19]:
from langchain_openai import AzureChatOpenAI

In [75]:
model = AzureChatOpenAI(
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment = os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version = os.environ["AZURE_OPENAI_API_VERSION"]
)

### Preprocess PDF

Installing Chroma for Vector DB support

In [57]:
!pip install -qU langchain_chroma

In [58]:
from langchain_chroma import Chroma
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

Split text, create embeddings and store in vector store (Load, Chunk and Index)

In [59]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
embeddings = AzureOpenAIEmbeddings(azure_endpoint=os.environ["AZURE_OPENAI_EMBEDDINGS_ENDPOINT"])

vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

retriever = vectorstore.as_retriever()

### Create RAG Pipeline

Import the helper functions for RAG pipeline

In [73]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

Prompt 1 -
```Contextualize the user's query based on chat history```

In [129]:
context_system_prompt = (
   "You are an assistant which has information on famous authors from english and hindi literature. When provided with a user query , you carefully look in the context provided and answer based on that. If the relevant information is not provided in the context, please respond saying the information is not provided in the context"

)
context_user_prompt = (
"Tell me more about Sci-fi movies? If you don't find relevant information from the given context then return the apology or not found message , don't return anything if not present in the given context"
    "{input}"
)


Template to guide model behaviour.

This template allows to retain context from chat history

In [130]:
messages = [
    ("system", context_system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", context_user_prompt),
]
prompt = ChatPromptTemplate.from_messages(messages)

Create ```create_history_aware_retriever``` to fetch relevant context from user's document based on the user's query and the conversation history

In [131]:
context_retriever = create_history_aware_retriever(
    llm=model, retriever=retriever, prompt=prompt
)

Prompt 2 - ```To generate the final answer using the relevant context.```

In [132]:
system_prompt = (
   "You are an assistant which has information on famous authors from english and hindi literature. When provided with a user query , you carefully look in the context provided and answer based on that. If the relevant information is not provided in the context, please respond saying the information is not provided in the context"
    "\n\n"
   "{context}"
)

In [117]:
system_prompt

'You are an assistant which has information on famous authors from english and hindi literature. When provided with a user query , you carefully look in the context provided and answer based on that. If the relevant information is not provided in the context, please respond saying the information is not provided in the context\n\n{context}'

Template to guide model behaviour.

This template allows to answer user's query with retrieved information

In [133]:
conversation_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", context_user_prompt),
    ]
)

Create a ```context_chain```
which "stuffs" the relevant documents into the language model's prompt, along with the user's query, and returns a response.

In [134]:
context_chain = create_stuff_documents_chain(
    model,
    conversation_prompt
)

Create RAG chain - serves two purposes

1 - Uses the ```context_retriever``` to find relevant text chunks from the vector database (Chroma).

2 - Passes the retrieved context and the question to the language model to generate an answer.

In [135]:
rag_chain = create_retrieval_chain(context_retriever,context_chain)

### Pass, Store and Update Chat History

Import packages to enable dynamic chat history support

In [136]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

```RunnableWithMessageHistory``` Wraps the RAG chain to maintain a history of the chat conversation.

```get_session_history``` is a helper function to create a ```ChatMessageHistory``` object for a new ```session_id``` or to retrieve ```ChatMessageHistory``` for an existing ```session_id```

In [137]:
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

### Invoking the conversational RAG chain
Presenting answer, and cofiguring a ```session_id``` to store unique chat history

In [138]:
conversational_rag_chain.invoke(
    {"input": "who is Charles Dicken?"},
    config={
        "configurable": {"session_id": "myuniqueid001"}
    },
)

{'input': 'who is Charles Dicken?',
 'chat_history': [],
 'context': [Document(metadata={'page': 0, 'source': '/content/authors/Charles_Dickens.pdf'}, page_content="Charles Dickens\nPortrait by Jeremiah Gurney, c. 1867–1868\nBorn Charles John Huffam Dickens\n7 February 1812\nPortsmouth, Hampshire,\nEngland\nDied 9 June 1870 (aged 58)\nHigham, Kent, England\nResting\nplace\nPoets' Corner,\nWestminster Abbey, England\n51°29′57″N 0°7′39″W\nOccupation Novelist\nNotable\nworks\nThe Pickwick Papers,\nOliver Twist,Nicholas Nickleby,\nA Christmas Carol,\nDavid Copperfield,\nBleak House,Little Dorrit,\nA Tale of Two Cities,\nGreat Expectations\nSpouse Catherine Thomson Hogarth  (m. 1836; sep. 1858) \nPartner Ellen Ternan (1857–1870, his\ndeath)\nCharles Dickens\nCharles John Huffam Dickens (/ˈdɪkɪnz/; 7 February\n1812 – 9 June 1870) was an English novelist,\njournalist, short story writer and social critic. He\ncreated some of literature's best-known fictional\ncharacters, and is regarded by ma

In [139]:
conversational_rag_chain.invoke(
    {"input": "Which rhythm would you give me to play Christmas Carol?"},
    config={
        "configurable": {"session_id": "myuniqueid001"}
    },
)["answer"]

"I'm sorry, but I don't have any information about the rhythm to play a Christmas Carol in the context provided."

In [140]:
conversational_rag_chain.invoke(
    {"input": "tell me who is william Shakespeare"},
    config={
        "configurable": {"session_id": "myuniqueid001"}
    },
)["answer"]

'William Shakespeare (c. 23 April 1564 – 23 April 1616) was an English playwright, poet, and actor. He is widely regarded as one of the greatest writers in the English language and the world\'s pre-eminent dramatist. Shakespeare is often referred to as England\'s national poet and the "Bard of Avon." His works, including plays, sonnets, and poems, have had a profound influence on literature and continue to be studied, performed, and celebrated worldwide. Some of his most famous plays include "Romeo and Juliet," "Hamlet," "Macbeth," "Othello," and "A Midsummer Night\'s Dream." Shakespeare\'s works explore various themes such as love, tragedy, comedy, and human nature. He was born and raised in Stratford-upon-Avon, England, and his legacy as a playwright and poet remains unparalleled.'

In [141]:
conversational_rag_chain.invoke(
    {"input": "what happened to Ratan Tata?"},
    config={
        "configurable": {"session_id": "myuniqueid001"}
    },
)["answer"]

"I'm sorry, but I don't have any information about what happened to Ratan Tata in the given context."

In [142]:
conversational_rag_chain.invoke(
    {"input": "Who is harivansh? is he died?"},
    config={
        "configurable": {"session_id": "myuniqueid002"}
    },
)["answer"]

'Harivansh Rai Bachchan, also known as Harivansh Rai Srivastava, was an Indian poet and writer. He was born on November 27, 1907, in Babupatti, United Provinces of Agra and Oudh, British India (present-day Uttar Pradesh, India). He passed away on January 18, 2003, in Mumbai, Maharashtra, India, at the age of 95. Harivansh Rai Bachchan was a renowned poet of the Nayi Kavita literary movement and a poet of the Hindi Kavi Sammelan. He is best known for his early work "Madhushala." He was also the father of Amitabh Bachchan.'

In [ ]:
store


{'myuniqueid001': InMemoryChatMessageHistory(messages=[HumanMessage(content='Which recipe would you give me if i only had carrots?', additional_kwargs={}, response_metadata={}), AIMessage(content="If you only have carrots, a simple and delicious recipe you can try is carrot soup. Here's a basic recipe:\n\nIngredients:\n- 4-5 large carrots, peeled and chopped\n- 1 onion, chopped\n- 2 cloves of garlic, minced\n- 4 cups vegetable or chicken broth\n- 1 teaspoon cumin powder\n- Salt and pepper to taste\n- 2 tablespoons olive oil\n- Optional toppings: fresh herbs, sour cream, croutons\n\nInstructions:\n1. Heat olive oil in a large pot over medium heat. Add onions and garlic and cook until softened.\n2. Add the chopped carrots to the pot and sauté for a few minutes.\n3. Pour in the vegetable or chicken broth and bring to a boil. Reduce heat to low, cover the pot, and simmer for about 20-25 minutes or until the carrots are tender.\n4. Using an immersion blender or regular blender, puree the so

In [143]:
while True:
  user_input = input("Enter 'q' to quit or anything else to continue: ")
  if user_input == 'q':
    break
  else:
    res = conversational_rag_chain.invoke(
    {"input": user_input},
    config={
        "configurable": {"session_id": "myuniqueid001"}
    },
    )["answer"]
    print(res)


Enter 'q' to quit or anything else to continue: who is chetan bhagat
Chetan Bhagat is an Indian author, columnist, screenwriter, and YouTuber. He was born on April 22, 1974, in New Delhi, India. Bhagat is known for his works in both English and Hindi languages. He has written novels in various genres including romance, realistic fiction, non-fiction, suspense, and thriller. Some of his notable works include "Five Point Someone," "2 States: The Story of My Marriage," and "The 3 Mistakes of My Life." Many of Bhagat's novels have been adapted into movies, such as "One Night @ the Call Center" and "Half Girlfriend." He has gained popularity for his relatable storytelling and his books have become bestsellers in India. In addition to his writing career, Bhagat has also been active as a columnist, writing for major Indian newspapers.
Enter 'q' to quit or anything else to continue: did he has any movie
I'm sorry, but I don't have information about Chetan Bhagat's involvement in sci-fi movies 